### Introduction


We plan to build a Python software library for automatic differentiation, a computational technique for evaluating derivatives that has several distinct advantages compared to alternative computer-based methods for differentiation.


Numerical methods, such as the finite differences method, are simple to implement but are plagued by floating point round-off errors and approximation errors, affecting accuracy and stability.

On the other hand, symbolic differentiation can compute derivatives up to machine precision, but can quickly become very computationally expensive.

As an alternative, automatic differentiation is capable of machine precision accuracy without the computational cost of symbolic differentiation.

This has significant implications for an extremely wide variety of   applications across science and engineering.

In our case, we plan to offer a software library that can compute the gradient or Jacobian for any mathematical function a user has implemented in Python, supporting scalar-functions of scalar values, vector functions of vectors, and scalar functions of vectors.

Users can expect to leverage our software for their own applications, such as optimization, root-finding, and much more.

### Background

#### Chain Rule

In calculus,  the derivative of the composition of two or more functions is defined as:

$$\dot{F}(x) = \dot{g}(x)\dot{f}(g(x))$$

In automatic differentiation, this rule is frequently utilized. All functions, even seemingly simple ones, use this rule. For example, $F(x) = sin(x)$ can be rewritten in the form $f(g(x))$ where $g(x)$ is simply $x$ and $f(x)$ is $sin(x)$. Thus, we have:


$$\dot{F}(x) = \dot{g}(x)\dot{f}(g(x))$$
$$\dot{F}(x)  = 1 *cos(x)$$

where 1 is $\dot{g}(x) = 1$ and $\dot{f}(x) = cos(x)$.

For a more complicated example, let us take $F(x) = sin(x^{2})$

$$ f(x) = sin(g(x))$$
$$ g(x) = x^{2}$$
$$ \dot{f}(x) = cos(g(x))$$
$$ \dot{g}(x) = 2x$$
$$\dot{F}(x) = \dot{g}(x)\dot{f}(g(x))$$
$$\dot{F}(x) = 2xcos(x^{2})$$

Using this rule, one can calculate the derivatives of increasinly complex functions, all while only needing to calculate the derivatives of a series of elementary functions.

####Computational Graph

The computation graph describes the order of calculations done to compute the derivative. For example consider the funcion $f\left(x, y, z\right) = \dfrac{1}{xyz} + \sin\left(\dfrac{1}{x} + \dfrac{1}{y} + \dfrac{1}{z}\right)$ where we want to evaluate the partial derivatives at (1, 2, 3). The table representation of the graph would look like this. Each row of the table represents one elementary step in the calculation. The function in each row is an elementary function on a combination of earlier rows, which lets us step by step build up the derivative by repeatedly applying the chain rule and at the same time we can also evaluate the function. The table can also be presented in a graph format, but this quickly becomes unwiedly for complicated functions. 


| Trace | Elementary Function | Current Value | Elementary Function Derivative | $\nabla_{x}$ Value  | $\nabla_{y}$ Value  | $\nabla_{z}$ Value  |
| :-------: | :-----------------: | :-----------: | :----------------------------: | :-----------------: | :-----------------: | :-----------------: |
| $x_{1} = x$ | $x_{1}$ | 1 | $\dot{x}_{1}$ | $1$ | $0$ | $0$ | 
| $x_{2} = y$ | $x_{2}$ | 2 | $\dot{x}_{2}$ | $0$ | $1$ | $0$ | 
| $x_{3} = z$ | $x_{3}$ | 3 | $\dot{x}_{3}$ | $0$ | $0$ | $1$ | 
| $x_{4}$ | $1/x_{1}$ | 1 | $-\dot{x}_{1}/x_{1}^{2}$ | $-1$ | $0$ | $0$ | 
| $x_{5}$ | $1/x_{2}$ | 1/2 | $-\dot{x}_{2}/x_{2}^{2}$ | $0$ | $-1/4$ | $0$ | 
| $x_{6}$ | $1/x_{3}$ | 1/3 | $-\dot{x}_{3}/x_{3}^{2}$ | $0$ | $0$ | $-1/9$ | 
| $x_{7}$ | $x_{4} * x_{5}$ | 1/2 | $\dot{x_4}x_5 + x_4 \dot{x_5}$ | $-1/2$ | $-1/4$ | $0$ | 
| $x_{8}$ | $x_{7} * x_{6}$ | 1/6 | $\dot{x_7}x_6 + x_7 \dot{x_6}$ | $-1/6$ | $-1/12$ | $-1/18$ | 
| $x_{9}$ | $x_{4} + x_{5} + x_{6}$ | 11/6 | $\dot{x_4} + \dot{x_5} + \dot{x_6}$ | $-1$ | $-1/4$ | $-1/9$ | 
| $x_{10}$ | $Sin(x_9)$ | 0.9657 | $\dot{x_9}Cos(x_9)$ | $0.2595$ | $-0.06488$ | $-0.02883$ | 
| $x_{11}$ | $x_{10} + x_{8} $ | 1.1324 | $\dot{x_{10}} + \dot{x_8}$ | $0.0928$ | $-0.0184$ | $-0.0267$ | 



#### Dual Numbers

Much like how imaginary numbers of the form $x+yi$ are an extension of the real plane that have some very useful properties, dual numbers are another interesting type of numbers of the from $x+y\epsilon$.

Similar to how we defined $i$ to have the property that $i^2 = -1$, the $\epsilon$ in dual numbers is defined such that $\epsilon^2 = 0$ (note: $\epsilon$ here is not 0!). A neat property of dual numbers can be illustrated with a very simple example.

Consider $(x+yi)^2$:
$$ (x+yi)^2 = x^2 + 2xyi + y^2i^2 = x^2 + 2xyi + y^2 $$
In particular, notice how the magnitude of the imaginary component ($y$) "feeds into" the real value of the result (the +y^2 contribution).

Now instead, consider $(x+y\epsilon)^2$:

$$ (x+y\epsilon)^2 = x^2  + 2xy\epsilon + y^2\epsilon^2 = x^2 + 2xy\epsilon (\text{recall:} \epsilon^2 = 0!) $$
Notice how with dual numbers, the magnitude of the dual component ($y$) _does not_ feed into the real value of the result.

Amongst other things, this quirk of the dual number makes it particularly suited for automatic differentiation. Consider the following:

Evaluate $y = x^2$ when $x\leftarrow x + \epsilon x^{\prime}$.

$y = (x + \epsilon x^{\prime})^2$

$y = (x^2 + 2 x x^{\prime} \epsilon + x^{\prime^2} \epsilon ^2)$

$y = (x^2 + 2 x x^{\prime} \epsilon)$

Notice here that by evaluating $y = x^2$ at $x\leftarrow x + \epsilon x^{\prime}$, we have calculated its derivative ($2x$)! Neat! 

In our implementation of automatic differentiation, dual numbers will be used to keep track of a function's value and it's derivative simulatenously.


#### Elementary Functions

In order to evaluate derivatives of a variety of different functions iteratively through the chain rule, we ultimately need to rely on a minimum set of so-called "elementary functions." These elementary functions consist of arithmetic operations (+, -, x, /) as well as atomic-sized, differentiable functions of a single variable that can be used as building blocks for more complex functions.



Here is a list of elementary functions:

* Addition, subtraction, multiplication, division
* Absolute Value
* Powers
* Roots
* Exponential
* Log
* Trigonometric Functions
* Inverse Trigonometric Functions



Explicitly defining the evaluation of these functions and their derivatives makes it possible to use the chain rule to evaluate the derivative of any differentiable function that is a combination of multiple elementary functions.

#### Seed Vectors


The use of seed vectors becomes apparent when considering more complicated examples. Consider a vector function $\textbf{f}(\textbf{x})$ that takes a vector input $\textbf{x} = (x_1, x_2)$:

$$\textbf{f(x)} = \begin{bmatrix} 2x_1+2x_2 \\ x_1x_2\end{bmatrix} $$

Let $\textbf{x}_{o} = (x_{o_1}, x_{o_2})$ be the outputs from the function. And define a directional derivative $D_{p}$ and seed vector $\textbf{p}$ such as follows:

$$D_{p}x_{o_i} = \sum_{j=1}^{2}{\dfrac{\partial x_{o_i}}{\partial x_{j}}p_{j}}$$

Computing the directional derivative for $\textbf{f}(\textbf{x})$:

$D_{p}x_{o_1} = {\dfrac{\partial x_{o_1}}{\partial x_{1}}p_{1}} + {\dfrac{\partial x_{o_1}}{\partial x_{2}}p_{2}}  = 2p_{1} + 2p_2$

$D_{p}x_{o_2} = {\dfrac{\partial x_{o_2}}{\partial x_{1}}p_{1}} + {\dfrac{\partial x_{o_2}}{\partial x_{2}}p_{2}} = x_2p_1 + x_1p_2$

This can be written more compactly as the Jacobian: $\textbf{J}=$
$
  \begin{bmatrix}
        2p_{1}   & 2p_2                        \\
        x_2p_1 & x_1p_2
      \end{bmatrix}
 $
 
Notice that by choosing different values for the seed vector $\textbf{p}$ allows us to select different components of the Jacobian! We can recover ${\dfrac{\partial x_{o}}{\partial x_{1}}}$ by choosing $\textbf{p} = (p_1, p_2) = (1,0)$, and recover ${\dfrac{\partial x_{o}}{\partial x_{2}}}$ by choosing $\textbf{p} = (0,1)$. We can also recover the full Jacobian by choosing $\textbf{p} = (1/\sqrt{2},1/\sqrt{2})$.

Typically, we would only be interested in the action of the Jacobian on some vector. The use of this seed vector allows us to compute the components of the Jacobian that we are interested in! We can however always still recover the full Jacobian by choosing an appropriate seed vector.

### How to Use PyDif



```
import pydif as pd
import numpy as np

def myfunc(x, y):
    return np.sin(x^2 * y )
    
myfunc_grad = pd.grad(myfunc)

//returns the gradient at the point
myfunc_grad(5, 1)

//returns the jacobian at the point 
myfunc_grad("jacobian")

```



### Software Organization
Our directory structure will look like this:

  ```
     pydif\
           pydif\
                 __init__.py
                 core.py
                 tests/
                      test_core.py
                      test_dual.py
                  dual/
                       __init__.py
                      dual.py
                  elementary/
                       __init__.py
                      elementary.py
                   
           README.md
           setup.py
           LICENSE
  ```
  
  tests will contain of our testing files. Which will be implemented using pytest. We will use TravisCI for continuous integration aswell as coveralls for coverage. core.py will contain the core of our package and is where we will actually implement automatic differentiation. duel.py will contain the duel class which will be imported in duel.py. elementary.py will contain the elementary functions. The project will be licensed under the MIT License.  

### Implementation

Our implementation will consist primarily of two classes: dual_number and grad. Dual_number will hold a value and derivative in a tuple. Grad will hold operators which are overloaded, as well as newly implemented operators. the grad class will allow a user to choose store the history of the derivatives and values in a list of tuples.  When it comes to elementary functions which are not overloadable, we will require that the user use our version. While this is not ideal, it will allow us to support dual numbers and allows us to circumvent the unoveraloadable nature of some elementary functions e.g trig functions. Each elementary function which is not overloadable will be wrapped in the elementary module. Elementary functions which are overloadable, e.g + * , will be implemented by writing the appropriate methods in the grad class. We are also going to implement a function to compute the Jacobian where necessary, such as when the user asks to be returned the jacbobian. 

We will also support multiple types of functions by looking at the type of the input. If a user passes an array like object then we know we have a scalar/vector to vector function, since the expectation is that if the user is passing an array that it will be an array of functions. We can also inspect the passed functions using the signature function from the inspect module to determine how many variables the function uses. 